# Monitor of Snakemake workflow

## Snakemake running example

```
INV_GEOMETRY=four_spheres_csf_3_mm
INV_MESH=normal
INV_DEGREE=2
FWD_GEOMETRY=${INV_GEOMETRY}
FWD_MESH=coarse
FWD_DEGREE=3
K=9

INV_PATH=${INV_GEOMETRY}/${INV_MESH}/${INV_DEGREE}
FWD_PATH=${FWD_GEOMETRY}/${FWD_MESH}/${FWD_DEGREE}

snakemake -j 1 \
FEM/solutions/paper/images/${INV_PATH}/kernels/${K}/images/${FWD_PATH}/fair_sources.csv \
FEM/solutions/paper/images/${INV_PATH}/kernels/${K}/images/${FWD_PATH}/kCSD_eigensources.csv
```

## Config

In [ ]:
SOURCES = 'fair_sources'
K=9

INV_GEOMETRY = 'four_spheres_csf_3_mm'
INV_MESH = 'normal'
INV_DEGREE = 2

FWD_GEOMETRY = INV_GEOMETRY
FWD_MESH = 'coarse'
FWD_DEGREE = 1

In [ ]:
INV_PATH=f'{INV_GEOMETRY}/{INV_MESH}/{INV_DEGREE}'
FWD_PATH=f'{FWD_GEOMETRY}/{FWD_MESH}/{FWD_DEGREE}'
PREFIX=f'FEM/solutions/paper/images/{INV_PATH}/kernels/{K}'

IMAGE_FILE = f'{PREFIX}/images/{FWD_PATH}/{SOURCES}.csv'

## Snakemake running

Run **the output** of the cell below in `kesi37` conda env.

In [ ]:
print(f"""snakemake -j 1 \\
{IMAGE_FILE}""")

In [ ]:
import collections
import itertools

import numpy as np
import pandas as pd

import kesi
import kesi._verbose as verbose
from kesi import common

%matplotlib inline
import matplotlib.pyplot as plt

from local import cbf

# kernel analysis

In [ ]:
methods = {}
_keys = ['eigenvalues',
         'eigenvectors',
         'eigensources',
         'lambdas',
         ]

for method in ['kCSD', 'kESI']:
    with np.load(f'{PREFIX}/{method}_analysis.npz') as fh:
        methods[method] = {k: fh[k.upper()] for k in _keys}
    
    for data in ['phi',
                 'kernel',
#                  'crosskernel',
                 ]:
        with np.load(f'{PREFIX}/{method}_{data}.npz') as fh:
            methods[method][data] = fh[data.upper()]

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

for method, matrices in methods.items():
    line = ax1.plot(matrices['eigenvalues'], label=method)[0]
    _EIGENVALUES = np.linalg.eigh(matrices['kernel'])[0][::-1]
    ax1.plot(_EIGENVALUES, color=line.get_color(), ls='none', marker='+')
    
    ax2.plot(matrices['eigenvalues'] / _EIGENVALUES - 1, color=line.get_color(), ls='none', marker='+')

ax1.set_yscale('log')
ax1.legend(loc='best')
ax1.set_ylabel('eigenvalues and squared singular values')
ax2.set_ylabel('relative difference between eigen- and squared singular values')

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
fig.suptitle('right singular vectors projected on eigenvectors')

for i, (method, matrices) in enumerate(methods.items(), start=1):
    _EIGENVECTORS = np.linalg.eigh(matrices['kernel'])[1][:, ::-1]
    _PROJECTION = np.matmul(matrices['eigenvectors'].T, _EIGENVECTORS)
    amp = abs(_PROJECTION).max()
    
    axes[0].plot(abs(np.diag(_PROJECTION)) - 1, label=method)
    
    axes[i].set_title(method)
    
    fig.colorbar(axes[i].imshow(_PROJECTION, vmin=-amp, vmax=amp, cmap=cbf.bwr),
                 ax=axes[i])
    
axes[0].legend(loc='best')
axes[0].set_ylabel('abs(diagonal) - 1')

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
fig.suptitle('left singular vectors projected on eigensources (from eigendecomposition)')

for i, (method, matrices) in enumerate(methods.items(), start=1):
    _EIGENVALUES, _EIGENVECTORS = np.linalg.eigh(matrices['kernel'])
    _EIGENVALUES, _EIGENVECTORS = _EIGENVALUES[::-1], _EIGENVECTORS[:, ::-1]
    _LAMBDAS = np.sqrt(_EIGENVALUES)
    _EIGENSOURCES = np.matmul(matrices['phi'],
                              np.matmul(_EIGENVECTORS,
                                        np.diag(1. / _LAMBDAS)))
    
    _PROJECTION = np.matmul(matrices['eigensources'].T, _EIGENSOURCES)
    amp = abs(_PROJECTION).max()
    
    axes[0].plot(abs(np.diag(_PROJECTION)) - 1, label=method)
    
    axes[i].set_title(method)
    
    fig.colorbar(axes[i].imshow(_PROJECTION, vmin=-amp, vmax=amp, cmap=cbf.bwr),
                 ax=axes[i])

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
fig.suptitle('left singular vectors projected on eigensources (sham eigendecomposition)')

for i, (method, matrices) in enumerate(methods.items(), start=1):
    _EIGENSOURCES = np.matmul(matrices['phi'],
                              np.matmul(matrices['eigenvectors'],
                                        np.diag(1. / matrices['lambdas'])))
    
    _PROJECTION = np.matmul(matrices['eigensources'].T, _EIGENSOURCES)
    amp = abs(_PROJECTION).max()
    
    axes[0].plot(abs(np.diag(_PROJECTION)) - 1, label=method)
    
    axes[i].set_title(method)
    
    fig.colorbar(axes[i].imshow(_PROJECTION, vmin=-amp, vmax=amp, cmap=cbf.bwr),
                 ax=axes[i])

In [ ]:
for method, matrices in methods.items():
    del matrices['phi'], matrices['eigensources']
    
    with np.load(f'{PREFIX}/{method}_crosskernel.npz') as fh:
        matrices['crosskernel'] = fh['CROSSKERNEL']

## CSD

In [ ]:
with np.load(f'{PREFIX}/{SOURCES}.npz') as fh:
    CSD_GRID = [fh[x] for x in ['X', 'Y', 'Z']]
    CSD = fh['CSD']

## IMAGES

In [ ]:
IMAGES = pd.read_csv(IMAGE_FILE)

In [ ]:
IMAGES

In [ ]:
plt.scatter(IMAGES.X, IMAGES.Z, marker='.')
plt.gca().add_artist(plt.Circle((0,0), radius=0.090, ls=':', edgecolor=cbf.BLACK, facecolor='none'))
plt.gca().add_artist(plt.Circle((0,0), radius=0.086, ls=':', edgecolor=cbf.VERMILION, facecolor='none'))
plt.gca().add_artist(plt.Circle((0,0), radius=0.082, ls=':', edgecolor=cbf.BLUE, facecolor='none'))
plt.gca().add_artist(plt.Circle((0,0), radius=0.079, ls=':', edgecolor=cbf.PURPLE, facecolor='none'))
plt.xlim(-0.03, 0.03)
plt.ylim(0.04, 0.08)
plt.gca().set_aspect('equal')

In [ ]:
for i in range(CSD.shape[-1]):
    V = IMAGES[f'SOURCE_{i}']
    plt.figure()
    plt.title(f'source #{i}')
    plt.axhline(1, ls=':', color=cbf.BLACK)
    plt.axhline(0, ls='--', color=cbf.BLACK)
    plt.axvline(i, ls=':', color=cbf.BLACK)
    plt.xlabel('eigensource')
    for method, marker in [('kCSD', '+'), ('kESI', 'x')]:
        plt.plot(np.matmul(V, methods[method]['eigenvectors']) / methods[method]['lambdas'],
                 ls='none',
                 marker=marker,
                 label=method)
    plt.legend(loc='best')

## Reconstruction errors

In [ ]:
norms = {'L1': lambda x: np.abs(x).mean(),
         'L2': lambda x: np.sqrt(np.square(x).mean()),
         'Linf': lambda x: np.abs(x).max(),
         }

def add_norms_to_dict(d, key_template, DATA):
    for name, norm in norms.items():
        d[key_template.format(name)] = norm(DATA)

In [ ]:
for method, matrices in methods.items():
    plt.plot(matrices['eigenvalues'], label=method)

plt.yscale('log')
plt.legend(loc='best')

In [ ]:
REGULARIZATION_PARAMETERS = np.logspace(3, 17, 5 * 14 + 1)

In [ ]:
es_reconstructors = {method: verbose.VerboseFFR._CrossKernelReconstructor(
                                                     kesi._engine._LinearKernelSolver(
                                                         matrices['kernel']),
                                                     np.matmul(np.diag(matrices['lambdas']),
                                                               matrices['eigenvectors'].T))
                     for method, matrices in methods.items()
                     }

In [ ]:
reconstructors = {method: verbose.VerboseFFR._CrossKernelReconstructor(
                                                   kesi._engine._LinearKernelSolver(
                                                       matrices['kernel']),
                                                   matrices['crosskernel'])
                  for method, matrices in methods.items()
                  }

In [ ]:
type(np.array(IMAGES.SOURCE_1))

In [ ]:
%%time
IMAGE_ERRORS = []
IMAGE_ERRORS_CV = []

for i in range(CSD.shape[-1]):
    print(i)
    _CSD_GT = CSD[:, :, :, i]
    V = np.array(IMAGES[f'SOURCE_{i}'])
    row = {'ES': i}
    add_norms_to_dict(row, 'GT_{}', _CSD_GT)
    row_cv = row.copy()
    IMAGE_ERRORS.append(row)
    IMAGE_ERRORS_CV.append(row_cv)


    for method, reconstructor in reconstructors.items():
        _ERRORS = common.cv(reconstructor, V, REGULARIZATION_PARAMETERS)
        for _row, _rp in [(row, 0),
                          (row_cv, REGULARIZATION_PARAMETERS[np.argmin(_ERRORS)])]:
            add_norms_to_dict(_row,
                              f'ERR_{method}_{{}}',
                              reconstructor(V, _rp) - _CSD_GT)

del _CSD_GT, _ERRORS

IMAGE_ERRORS = pd.DataFrame(IMAGE_ERRORS)
IMAGE_ERRORS_CV = pd.DataFrame(IMAGE_ERRORS_CV)

## Reconstruction error plots (no regularization)

In [ ]:
for norm in norms:
    plt.figure()
    plt.title(norm)
    for method, color, ls in [('kCSD', cbf.SKY_BLUE, '-'),
                              ('kESI', cbf.VERMILION, ':'),
                             ]:
        plt.plot(IMAGE_ERRORS.ES,
                 IMAGE_ERRORS[f'ERR_{method}_{norm}'] / IMAGE_ERRORS[f'GT_{norm}'],
                 color=color,
                 ls=ls,
                 label=method)
    plt.legend(loc='best')
    plt.axhline(1, color=cbf.BLACK, ls=':')
    plt.axhline(0.5, color=cbf.BLACK, ls=':')
    plt.axhline(0.1, color=cbf.BLACK, ls=':')
    plt.axhline(0.05, color=cbf.BLACK, ls=':')
    plt.yscale('log')

## Reconstruction error plots (regularization)

In [ ]:
for norm in norms:
    plt.figure()
    plt.title(norm)
    for method, color, ls in [('kCSD', cbf.SKY_BLUE, '-'),
                              ('kESI', cbf.VERMILION, ':'),
                             ]:
        plt.plot(IMAGE_ERRORS_CV.ES,
                 IMAGE_ERRORS_CV[f'ERR_{method}_{norm}'] / IMAGE_ERRORS_CV[f'GT_{norm}'],
                 color=color,
                 ls=ls,
                 label=method)
    plt.legend(loc='best')
    plt.axhline(1, color=cbf.BLACK, ls=':')
    plt.axhline(0.5, color=cbf.BLACK, ls=':')
    plt.axhline(0.1, color=cbf.BLACK, ls=':')
    plt.axhline(0.05, color=cbf.BLACK, ls=':')
    plt.yscale('log')

## compare approaches

In [ ]:
for norm in norms:
    plt.figure()
    plt.title(norm)
    for method, color in [('kCSD', cbf.SKY_BLUE),
                          ('kESI', cbf.VERMILION),
                         ]:
        plt.plot(IMAGE_ERRORS.ES,
                 IMAGE_ERRORS[f'ERR_{method}_{norm}'] / IMAGE_ERRORS[f'GT_{norm}'],
                 color=color,
                 ls='-',
                 label=method)
        plt.plot(IMAGE_ERRORS_CV.ES,
                 IMAGE_ERRORS_CV[f'ERR_{method}_{norm}'] / IMAGE_ERRORS_CV[f'GT_{norm}'],
                 color=color,
                 ls=':',
                 label=f'{method} (CV)')

    plt.legend(loc='best')
    plt.axhline(1, color=cbf.BLACK, ls=':')
    plt.axhline(0.5, color=cbf.BLACK, ls=':')
    plt.axhline(0.1, color=cbf.BLACK, ls=':')
    plt.axhline(0.05, color=cbf.BLACK, ls=':')
    plt.yscale('log')

In [ ]:
class CardinalPlaneVisualisation(object):
    SPHERE_RADII = [0.079, 0.082, 0.086, 0.090]
    SPHERE_RADII_GT = [0.079, 0.080, 0.085, 0.090]

    def __init__(self,
                 grid,
                 plane_intersection,
                 dpi=17,
                 cmap=cbf.bwr,
                 amp=None,
                 length_factor=1,
                 length_unit='$m$',
                 unit_factor=1,
                 unit=''):
        self.grid = grid
        self.plane_intersection = np.array(plane_intersection)
        self.indices = [np.searchsorted(g, a)
                        for a, g in zip(plane_intersection,
                                        grid)]
        self.dpi = dpi
        self.cmap = cmap
        self.amp = amp
        self.length_factor = length_factor
        self.length_unit = length_unit
        self.unit_factor = unit_factor
        self.unit = unit

    def start_new_image(self, title, wx, wy, wz):
        self.fig = plt.figure(figsize=((wx + wy) / self.dpi,
                                       (wz + wy) / self.dpi))
        if title is not None:
            self.fig.suptitle(title)

        gs = plt.GridSpec(2, 2,
                          figure=self.fig,
                          width_ratios=[wx, wy],
                          height_ratios=[wz, wy])

        self.ax_xz = self.fig.add_subplot(gs[0, 0])
        self.ax_xz.set_aspect('equal')
        self.ax_xz.set_ylabel(f'Z [{self.length_unit}]')
        self.ax_xz.set_xlabel(f'X [{self.length_unit}]')

        self.ax_yx = self.fig.add_subplot(gs[1, 1])
        self.ax_yx.set_aspect('equal')
        self.ax_yx.set_ylabel(f'X [{self.length_unit}]')
        self.ax_yx.set_xlabel(f'Y [{self.length_unit}]')

        self.ax_yz = self.fig.add_subplot(gs[0, 1],
                                          sharey=self.ax_xz,
                                          sharex=self.ax_yx)
        self.ax_yz.set_aspect('equal')

        self.cax = self.fig.add_subplot(gs[1, 0])
        self.cax.set_visible(False)

    def finish_image(self):
        x, y, z = self.length_factor * self.plane_intersection

        self.ax_xz.axvline(x, ls=':', color=cbf.BLACK)
        self.ax_xz.axhline(z, ls=':', color=cbf.BLACK)

        self.ax_yx.axvline(y, ls=':', color=cbf.BLACK)
        self.ax_yx.axhline(x, ls=':', color=cbf.BLACK)

        self.ax_yz.axvline(y, ls=':', color=cbf.BLACK)
        self.ax_yz.axhline(z, ls=':', color=cbf.BLACK)
        self.fig.colorbar(self.im, ax=self.cax,
                          orientation='horizontal',
                          label=self.unit)

    def plot_volume(self, DATA, title=None, amp=None):
        self.start_new_image(title, *DATA.shape)
        ix, iy, iz = self.indices
        self._plot_planes([DATA[ix:ix+1, :, :],
                           DATA[:, iy:iy+1, :],
                           DATA[:, :, iz:iz+1],
                           ],
                           amp if amp is not None else abs(DATA).max())
        self.finish_image()

    def _plot_planes(self, DATA_PLANES, amp):
        DATA_ZY = DATA_PLANES[0][0, :, :].T * self.unit_factor
        DATA_ZX = DATA_PLANES[1][:, 0, :].T * self.unit_factor
        DATA_XY = DATA_PLANES[2][:, :, 0] * self.unit_factor
        
        def _extent(first, second):
            _first = self.grid[first] * self.length_factor
            _second = self.grid[second] * self.length_factor
            return (_first.min(), _first.max(),
                    _second.min(), _second.max())

        self.ax_xz.imshow(DATA_ZX,
                          vmin=-amp * self.unit_factor,
                          vmax=amp * self.unit_factor,
                          cmap=self.cmap,
                          origin='lower',
                          extent=_extent(0, 2))
        self.ax_yx.imshow(DATA_XY,
                          vmin=-amp * self.unit_factor,
                          vmax=amp * self.unit_factor,
                          cmap=self.cmap,
                          origin='lower',
                          extent=_extent(1, 0))
        self.im = self.ax_yz.imshow(DATA_ZY,
                                    vmin=-amp * self.unit_factor,
                                    vmax=amp * self.unit_factor,
                                    cmap=self.cmap,
                                    origin='lower',
                                    extent=_extent(1, 2))

    def plot_planes(self,
                    DATA_PLANES,
                    title=None,
                    amp=None):

        DATA_YZ, DATA_XZ, DATA_XY = DATA_PLANES
        wx, wy, _ = DATA_XY.shape
        wz = DATA_YZ.shape[2]
        assert DATA_YZ.shape[1] == wy
        assert DATA_XZ.shape[0] == wx
        assert DATA_XZ.shape[2] == wz
        
        self.start_new_image(title, wx, wy, wz)
        self._plot_planes(DATA_PLANES,
                          amp if amp is not None else max(abs(_A).max() for _A in DATA_PLANES))
        self.finish_image()

    def compare_with_gt(self, GT, CSD, title=''):
        ERROR = CSD - GT
        error_L2 = np.sqrt(np.square(ERROR).sum() / np.square(GT_CSD).sum())
        amp = max(abs(CSD).max(),
                  abs(GT).max(),
                  abs(ERROR).max())
        self.plot_volume(GT,
                         title='GT CSD',
                         amp=amp)
        self._add_spheres(self.SPHERE_RADII_GT)
        self.plot_volume(CSD,
                         title=f'{title} reconstruction',
                         amp=amp)
        self._add_spheres(self.SPHERE_RADII)
        self.plot_volume(ERROR,
                         title=f'{title} error (GT normalized L2 norm: {error_L2:.2g})',
                         amp=amp)

    def _add_spheres(self, sphere_radii):
        for c, ax in zip(self.plane_intersection,
                         [self.ax_yz,
                          self.ax_xz,
                          self.ax_yx]):
            for r2 in np.square(sphere_radii):
                self._plot_circle(ax, np.sqrt(r2 - np.square(c)))

    def _plot_circle(self, ax, r):
        ax.add_artist(plt.Circle((0, 0), r * self.length_factor,
                                facecolor='none',
                                edgecolor=cbf.BLACK,
                                linestyle=':'))

    @property
    def PLANES_XYZ(self):
        return [[[c] if i == j else A for j, A in enumerate(self.grid)]
                for i, c in enumerate(self.plane_intersection)]

In [ ]:
csd_plotter = CardinalPlaneVisualisation([_x.flatten() for _x in CSD_GRID],
                                         [0, 0, 0.058],
                                         unit_factor=1e-3,
                                         unit='$\\frac{pA}{mm^3}$',
                                         length_factor=1e3,
                                         length_unit='$mm$')

In [ ]:
methods.keys()

In [ ]:
# _ES = 1
_ES = 2
# _ES = 7

csd = {'GT': CSD[:, :, :, _ES]}
_V = np.array(IMAGES[f'SOURCE_{_ES}'])

plt.figure()
plt.title('CV')
plt.xscale('log')
plt.yscale('log')

for method, reconstructor in reconstructors.items():
    _ERRORS = common.cv(reconstructor, _V, REGULARIZATION_PARAMETERS)
    regularization_parameter = REGULARIZATION_PARAMETERS[np.argmin(_ERRORS)]
    _l = plt.plot(REGULARIZATION_PARAMETERS, _ERRORS, label=method)
    plt.axvline(regularization_parameter,
                ls=':',
                color=_l[0].get_color())

    csd[method] = reconstructor(_V, regularization_parameter)

plt.legend(loc='best')

In [ ]:
for method, _CSD in csd.items():
    csd_plotter.plot_volume(_CSD, method)
    csd_plotter._add_spheres(csd_plotter.SPHERE_RADII)

    csd_plotter.ax_xz.set_xlim(-20, 20)
    csd_plotter.ax_yx.set_xlim(-20, 20)
    csd_plotter.ax_yz.set_xlim(-20, 20)
    
    csd_plotter.ax_xz.set_ylim(25, 90)
    csd_plotter.ax_yx.set_ylim(-20, 20)
    csd_plotter.ax_yz.set_ylim(25, 90)
    
    csd_plotter.ax_xz.scatter(IMAGES.X * 1e3, IMAGES.Z * 1e3, marker='x', color=cbf.BLACK)
    csd_plotter.ax_yx.scatter(IMAGES.Y * 1e3, IMAGES.X * 1e3, marker='x', color=cbf.BLACK)
    csd_plotter.ax_yz.scatter(IMAGES.Y * 1e3, IMAGES.Z * 1e3, marker='x', color=cbf.BLACK)

In [ ]:
for _ES in range(CSD.shape[-1]):
    print(_ES)
    
    csd = {f'ES #{_ES} GT': CSD[:, :, :, _ES]}
    _V = np.array(IMAGES[f'SOURCE_{_ES}'])

#     plt.figure()
#     plt.title('CV')
#     plt.xscale('log')
#     plt.yscale('log')

    for method, reconstructor in reconstructors.items():
        _ERRORS = common.cv(reconstructor, _V, REGULARIZATION_PARAMETERS)
        regularization_parameter = REGULARIZATION_PARAMETERS[np.argmin(_ERRORS)]
#         _l = plt.plot(REGULARIZATION_PARAMETERS,
#                       _ERRORS,
#                      label=method)
#         plt.axvline(regularization_parameter,
#                     ls=':',
#                     color=_l[0].get_color())

        csd[f'ES #{_ES} {method}'] = reconstructor(_V, regularization_parameter)
#     plt.legend(loc='best')

    for method, _CSD in csd.items():
        csd_plotter.plot_volume(_CSD, method)
        csd_plotter._add_spheres(csd_plotter.SPHERE_RADII)

        csd_plotter.ax_xz.set_xlim(-20, 20)
        csd_plotter.ax_yx.set_xlim(-20, 20)
        csd_plotter.ax_yz.set_xlim(-20, 20)

        csd_plotter.ax_xz.set_ylim(25, 90)
        csd_plotter.ax_yx.set_ylim(-20, 20)
        csd_plotter.ax_yz.set_ylim(25, 90)

        csd_plotter.ax_xz.scatter(IMAGES.X * 1e3, IMAGES.Z * 1e3, marker='x', color=cbf.BLACK)
        csd_plotter.ax_yx.scatter(IMAGES.Y * 1e3, IMAGES.X * 1e3, marker='x', color=cbf.BLACK)
        csd_plotter.ax_yz.scatter(IMAGES.Y * 1e3, IMAGES.Z * 1e3, marker='x', color=cbf.BLACK)

## Noisy reconstruction

In [ ]:
%%time
NOISE_LEVELS = [0, 5, 10, 20, 50, 100, 200]

NOISY_IMAGE_ERRORS_CV = []
NOISY_IMAGE_ERRORS = []

np.random.seed(42)

for i in range(CSD.shape[-1]):
    print(i)
    
    _CSD_GT = CSD[:, :, :, i]
    V = np.array(IMAGES[f'SOURCE_{i}'])

    row_cv = {'ES': i}
    add_norms_to_dict(row_cv, 'GT_{}', _CSD_GT)

    row = row_cv.copy()
    NOISY_IMAGE_ERRORS_CV.append(row_cv)
    NOISY_IMAGE_ERRORS.append(row)
    
    for noise_level in NOISE_LEVELS:
        NOISE = np.random.normal(scale=V.std() * noise_level / 100.,
                                 size=len(V))
        NOISY_V = V + NOISE
        
        for method, reconstructor in reconstructors.items():
            _ERRORS = common.cv(reconstructor, NOISY_V, REGULARIZATION_PARAMETERS)

            for _row, _rp in [(row, 0),
                              (row_cv, REGULARIZATION_PARAMETERS[np.argmin(_ERRORS)])]:
                add_norms_to_dict(_row,
                                  f'ERR_{method}_NOISE_{noise_level}_STD_{{}}',
                                  reconstructor(NOISY_V, _rp) - _CSD_GT)

NOISY_IMAGE_ERRORS_CV = pd.DataFrame(NOISY_IMAGE_ERRORS_CV)
NOISY_IMAGE_ERRORS = pd.DataFrame(NOISY_IMAGE_ERRORS)

del _CSD_GT

In [ ]:
NOISE_LS = {0: '-',
            5: '--',
            10: '-.',
            20: ':',
#             50: '-',
#             100: '--',
#             200: '-.',
            }

for regularization, DF in [('none', NOISY_IMAGE_ERRORS),
                           ('CV', NOISY_IMAGE_ERRORS_CV)]:
    for norm in norms:
        plt.figure(figsize=(16, 12))
        plt.title(f'{norm} error; regularization: {regularization}')

        for method, color in [('kCSD', cbf.SKY_BLUE),
                              ('kESI', cbf.VERMILION),
                             ]:
            for noise_level, ls in NOISE_LS.items():
                plt.plot(DF.ES,
                         DF[f'ERR_{method}_NOISE_{noise_level}_STD_{norm}'] / DF[f'GT_{norm}'],
                         color=color,
                         ls=ls,
                         label=f'{method} ({noise_level}% noise)')

        plt.legend(loc='best')
        plt.axhline(1, color=cbf.BLACK, ls=':')
        plt.axhline(0.5, color=cbf.BLACK, ls=':')
        plt.axhline(0.1, color=cbf.BLACK, ls=':')
        plt.axhline(0.05, color=cbf.BLACK, ls=':')
        plt.yscale('log')

In [ ]:
%%time
N_NOISE = 100
NOISE_LEVEL = 10
NOISY_IMAGE_ERRORS_CV_Q = []
NOISY_IMAGE_ERRORS_Q = []

np.random.seed(42)
NOISE = np.random.normal(scale=NOISE_LEVEL / 100.,
                         size=(N_NOISE, CSD.shape[-1]))

for i in range(CSD.shape[-1]):
    print(i)
    
    _CSD_GT = CSD[:, :, :, i]
    V = np.array(IMAGES[f'SOURCE_{i}'])

    row_cv = collections.defaultdict(list,
         {'ES': i})
    add_norms_to_dict(row_cv, 'GT_{}', _CSD_GT)

    row = row_cv.copy()
    NOISY_IMAGE_ERRORS_CV_Q.append(row_cv)
    NOISY_IMAGE_ERRORS_Q.append(row)

#     for _NOISE in NOISE * np.sqrt(np.square(V).sum()):
    for _NOISE in NOISE * V.std():
        NOISY_V = V + _NOISE


        for method, reconstructor in reconstructors.items():
            _ERRORS = common.cv(reconstructor, NOISY_V, REGULARIZATION_PARAMETERS)

            for _row, _rp in [(row, 0),
                              (row_cv, REGULARIZATION_PARAMETERS[np.argmin(_ERRORS)])]:
                _tmp = {}
                add_norms_to_dict(_tmp,
                                  f'ERR_{method}_{{}}',
                                  reconstructor(NOISY_V, _rp) - _CSD_GT)
                for k, v in _tmp.items():
                    _row[k].append(v)
                  
    for _row, method, norm in itertools.product([row, row_cv],
                                                ['kCSD', 'kESI'],
                                                norms):
        _ERRORS = _row[f'ERR_{method}_{norm}']
        for q in [25, 50,
                  100, 250, 500, 750, 900,
                  950, 975,
                  ]:
            _row[f'ERR_{method}_{norm}_{q:03d}'] = np.quantile(_ERRORS, q * 1e-3)

        _row[f'ERR_{method}_{norm}'] = norms[norm](_ERRORS)

NOISY_IMAGE_ERRORS_CV_Q = pd.DataFrame(NOISY_IMAGE_ERRORS_CV_Q)
NOISY_IMAGE_ERRORS_Q = pd.DataFrame(NOISY_IMAGE_ERRORS_Q)

del _CSD_GT

In [ ]:
for norm in norms:
    plt.figure()
    plt.title(norm)
    for method, color in [('kCSD', cbf.SKY_BLUE),
                          ('kESI', cbf.VERMILION),
                         ]:
        plt.plot(NOISY_IMAGE_ERRORS_Q.ES,
                 NOISY_IMAGE_ERRORS_Q[f'ERR_{method}_{norm}'] / NOISY_IMAGE_ERRORS_Q[f'GT_{norm}'],
                 color=color,
                 ls='-',
                 label=method)
        plt.plot(NOISY_IMAGE_ERRORS_CV_Q.ES,
                 NOISY_IMAGE_ERRORS_CV_Q[f'ERR_{method}_{norm}'] / NOISY_IMAGE_ERRORS_CV_Q[f'GT_{norm}'],
                 color=color,
                 ls=':',
                 label=f'{method} (CV)')

    plt.legend(loc='best')
    plt.axhline(1, color=cbf.BLACK, ls=':')
    plt.axhline(0.5, color=cbf.BLACK, ls=':')
    plt.axhline(0.1, color=cbf.BLACK, ls=':')
    plt.axhline(0.05, color=cbf.BLACK, ls=':')
    plt.yscale('log')

In [ ]:
BOTTOM = 250
TOP = 750
# BOTTOM = 25
# TOP = 975

for norm in norms:
    plt.figure()
    plt.title(f'Noisy ({norm}; {(TOP - BOTTOM) / 10:g}%CI)')
    for method, color in [('kCSD', cbf.SKY_BLUE),
                          ('kESI', cbf.VERMILION),
                         ]:
        for q in [TOP, BOTTOM]:
            plt.plot(NOISY_IMAGE_ERRORS_Q.ES,
                     NOISY_IMAGE_ERRORS_Q[f'ERR_{method}_{norm}_{q:03d}'] / NOISY_IMAGE_ERRORS_Q[f'GT_{norm}'],
                     color=color,
                     ls='-',
                     label=method)
            plt.plot(NOISY_IMAGE_ERRORS_CV_Q.ES,
                     NOISY_IMAGE_ERRORS_CV_Q[f'ERR_{method}_{norm}_{q:03d}'] / NOISY_IMAGE_ERRORS_CV_Q[f'GT_{norm}'],
                     color=color,
                     ls=':',
                     label=f'{method} (CV)')

    plt.legend(loc='best')
    plt.axhline(1, color=cbf.BLACK, ls=':')
    plt.axhline(0.5, color=cbf.BLACK, ls=':')
    plt.axhline(0.1, color=cbf.BLACK, ls=':')
    plt.axhline(0.05, color=cbf.BLACK, ls=':')
    plt.yscale('log')